In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')

In [3]:
def geometric_mean(x):
    return np.exp(np.log(x[x>0]).mean())

In [4]:
app_train_origin =  pd.read_csv('./home-credit-default-risk/application_train.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/application_test.csv')
app_train_origin.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
display(app_train['DAYS_EMPLOYED'].value_counts())
print("正の値の割合: {:.4f}".format((app_train['DAYS_EMPLOYED']>0).mean()))
print("正の値の個数: {}".format((app_train['DAYS_EMPLOYED']>0).sum()))

 365243    55374
-200         156
-224         152
-230         151
-199         151
           ...  
-13961         1
-11827         1
-10176         1
-9459          1
-8694          1
Name: DAYS_EMPLOYED, Length: 12574, dtype: int64

正の値の割合: 0.1801
正の値の個数: 55374


### one-hot-encoding

In [7]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
### CODE_GENDER
app_train['CODE_GENDER'].replace(['XNA'], ['M'], inplace=True)
app_test['CODE_GENDER'].replace(['XNA'], ['M'], inplace=True)
app_train['CODE_GENDER'].replace(['F', 'M'], [0, 1], inplace=True)
app_test['CODE_GENDER'].replace(['F', 'M'], [0, 1], inplace=True)
#encoding_columns.remove('CODE_GENDER')
### NAME_INCOME_TYPE
app_train['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
app_test['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
app_train['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
app_test['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
#encoding_columns.remove('NAME_INCOME_TYPE')
### NAME_FAMILY_STATUS
app_train['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
app_test['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
app_train['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)
app_test['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)
#encoding_columns.remove('NAME_FAMILY_STATUS')

In [9]:
# one-hotencoding
encoding_columns = []

for col in app_test.columns:
    if app_test[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns.append('CODE_GENDER')
encoding_columns.append('NAME_INCOME_TYPE')
encoding_columns.append('NAME_FAMILY_STATUS')

NAME_CONTRACT_TYPE
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_EDUCATION_TYPE
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [10]:
for col in encoding_columns:
    app_train = pd.get_dummies(app_train, columns=[col], dummy_na=True)
    app_test = pd.get_dummies(app_test, columns=[col], dummy_na=True)
    
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_INCOME_TYPE_4.0,NAME_INCOME_TYPE_5.0,NAME_INCOME_TYPE_6.0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0.0,NAME_FAMILY_STATUS_1.0,NAME_FAMILY_STATUS_2.0,NAME_FAMILY_STATUS_3.0,NAME_FAMILY_STATUS_4.0,NAME_FAMILY_STATUS_nan
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,1,0,0,0,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,1,0,0,0,0,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,1,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,1,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,1,0,0,0,0


In [11]:
### 
for col in app_train.columns:
    app_train[col] = app_train[col].replace('\s', '_', regex=True)
    app_train[col] = app_train[col].replace(':', '_', regex=True)
    app_train[col] = app_train[col].replace('/', '_', regex=True)
    
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_INCOME_TYPE_4.0,NAME_INCOME_TYPE_5.0,NAME_INCOME_TYPE_6.0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0.0,NAME_FAMILY_STATUS_1.0,NAME_FAMILY_STATUS_2.0,NAME_FAMILY_STATUS_3.0,NAME_FAMILY_STATUS_4.0,NAME_FAMILY_STATUS_nan
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,1,0,0,0,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,1,0,0,0,0,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,1,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,1,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,1,0,0,0,0


In [12]:
### 
for col in app_test.columns:
    app_test[col] = app_test[col].replace('\s', '_', regex=True)
    app_test[col] = app_test[col].replace(':', '_', regex=True)
    app_test[col] = app_test[col].replace('/', '_', regex=True)
    
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_INCOME_TYPE_4.0,NAME_INCOME_TYPE_5.0,NAME_INCOME_TYPE_6.0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0.0,NAME_FAMILY_STATUS_1.0,NAME_FAMILY_STATUS_2.0,NAME_FAMILY_STATUS_3.0,NAME_FAMILY_STATUS_4.0,NAME_FAMILY_STATUS_nan
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,1,0,0,0,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,1,0,0,0,0,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,1,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,1,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,1,0,0,0,0


In [13]:
len(app_train.columns)

259

In [14]:
len(app_test.columns)

258

In [15]:
app_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_INCOME_TYPE_4.0,NAME_INCOME_TYPE_5.0,NAME_INCOME_TYPE_6.0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0.0,NAME_FAMILY_STATUS_1.0,NAME_FAMILY_STATUS_2.0,NAME_FAMILY_STATUS_3.0,NAME_FAMILY_STATUS_4.0,NAME_FAMILY_STATUS_nan
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,...,0,0,0,0,1,0,0,0,0,0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,...,0,0,0,0,1,0,0,0,0,0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,...,0,0,0,0,1,0,0,0,0,0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,...,0,0,0,0,1,0,0,0,0,0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,...,0,0,0,0,1,0,0,0,0,0


In [16]:
app_train = app_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))
app_test = app_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### app特徴量作成

In [17]:
app_train['DAYS_EMPLOYED'] = app_train['DAYS_EMPLOYED'].replace(365243, np.nan)

In [18]:
docs = [_f for _f in app_train.columns if 'FLAG_DOC' in _f]
live = [_f for _f in app_train.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
app_train['ORGANIZATION_TYPE'] = app_train_origin['ORGANIZATION_TYPE'].copy()

# nullに置き換え
app_train['DAYS_EMPLOYED'] = app_train['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = app_train[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
app_train['INCOME_div_PERSON'] = app_train['AMT_INCOME_TOTAL'] / app_train['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
app_train['INCOME_div_EMPLOYED'] = app_train['AMT_INCOME_TOTAL'] / app_train['DAYS_EMPLOYED']

#
app_train['NEW_CREDIT_TO_ANNUITY_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_ANNUITY']

app_train['NEW_CREDIT_TO_GOODS_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_GOODS_PRICE']

app_train['NEW_DOC_IND_KURT'] = app_train[docs].kurtosis(axis=1)

app_train['NEW_LIVE_IND_SUM'] = app_train[live].sum(axis=1)

app_train['NEW_INC_PER_CHLD'] = app_train['AMT_INCOME_TOTAL'] / (1 + app_train['CNT_CHILDREN'])

app_train['NEW_INC_BY_ORG'] = app_train['ORGANIZATION_TYPE'].map(inc_by_org)

app_train['NEW_EMPLOY_TO_BIRTH_RATIO'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

app_train['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = app_train['DAYS_EMPLOYED'] / (app_train['DAYS_BIRTH'] + 18*365)

app_train['NEW_BIRTH_TO_EMPLOY_RATIO'] = app_train['DAYS_BIRTH'] / (1 + app_train['DAYS_EMPLOYED'])

app_train['NEW_INCOME_TO_ANNUITY_RATIO'] = app_train['AMT_INCOME_TOTAL'] / (1 + app_train['AMT_ANNUITY'])

app_train['NEW_ANNUITY_TO_INCOME_RATIO'] = app_train['AMT_ANNUITY'] / (1 + app_train['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
app_train['NEW_EXT_SOURCES_MEDIAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

app_train['EXT_SOURCE_mean'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
app_train['EXT_SOURCE_max'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

app_train['EXT_SOURCE_min'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_train['EXT_SOURCE_std'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

app_train['EXT_SOURCE_count'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

app_train['NEW_EXT_SOURCES_PROD'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

app_train['NEW_EXT_SOURCES_MAD'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

app_train['NEW_EXT_SOURCES_GEO'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
app_train['DAYS_EMPLOYED_div_BERTH'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
app_train['ANNUITY_div_INCOME'] = app_train['AMT_ANNUITY'] / app_train['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
app_train['ANNUITY_div_CREDIT'] = app_train['AMT_ANNUITY'] / app_train['AMT_CREDIT']

### 
app_train['credit_goods_price_ratio'] = app_train['AMT_CREDIT'] / app_train['AMT_GOODS_PRICE']
 
app_train['credit_downpayment'] = app_train['AMT_GOODS_PRICE'] - app_train['AMT_CREDIT']

app_train['USER_AGE'] = abs(app_train['DAYS_BIRTH'])/364.25

app_train['NEW_CAR_TO_BIRTH_RATIO'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_BIRTH']

app_train['NEW_CAR_TO_EMPLOY_RATIO'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_EMPLOYED']

app_train['NEW_PHONE_TO_BIRTH_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_BIRTH']

app_train['NEW_PHONE_TO_EMPLOYED_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_EMPLOYED']

app_train['NEW_CREDIT_TO_INCOME_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_INCOME_TOTAL']

app_train['NEW_INCOME_CREDIT_PERC'] = app_train['AMT_INCOME_TOTAL'] / app_train['AMT_CREDIT']

app_train['NEW_INCOME_PER_PERSON'] = app_train['AMT_INCOME_TOTAL'] / app_train['CNT_FAM_MEMBERS']

### 2022/06/25
app_train['app_EXT_SOURCE_prod'] = app_train['EXT_SOURCE_1'] * app_train['EXT_SOURCE_2'] * app_train['EXT_SOURCE_3']
app_train['app_EXT_SOURCE_1_times_EXT_SOURCE_2'] = app_train['EXT_SOURCE_1'] * app_train['EXT_SOURCE_2']
app_train['app_EXT_SOURCE_1_times_EXT_SOURCE_3'] = app_train['EXT_SOURCE_1'] * app_train['EXT_SOURCE_3']
app_train['app_EXT_SOURCE_2_times_EXT_SOURCE_3'] = app_train['EXT_SOURCE_2'] * app_train['EXT_SOURCE_3']
app_train['app_EXT_SOURCE_1_times_DAYS_EMPLOYED'] = app_train['EXT_SOURCE_1'] * app_train['DAYS_EMPLOYED']
app_train['app_EXT_SOURCE_2_times_DAYS_EMPLOYED'] = app_train['EXT_SOURCE_2'] * app_train['DAYS_EMPLOYED']
app_train['app_EXT_SOURCE_3_times_DAYS_EMPLOYED'] = app_train['EXT_SOURCE_3'] * app_train['DAYS_EMPLOYED']
app_train['app_EXT_SOURCE_1_PER_DAYS_BIRTH'] = app_train['EXT_SOURCE_1'] / app_train['DAYS_BIRTH']
app_train['app_EXT_SOURCE_2_PER_DAYS_BIRTH'] = app_train['EXT_SOURCE_2'] / app_train['DAYS_BIRTH']
app_train['app_EXT_SOURCE_3_PER_DAYS_BIRTH'] = app_train['EXT_SOURCE_3'] / app_train['DAYS_BIRTH']

### 2022/06/26
#app_train['app_most_popular_AMT_GOODS_PRICE'] = app_train['AMT_GOODS_PRICE'] \
#    .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
#app_train['app_popular_AMT_GOODS_PRICE'] = app_train['AMT_GOODS_PRICE'] \
#    .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
#app_train['app_OWN_CAR_AGE_div_DAYS_BIRTH'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_BIRTH']
#app_train['app_OWN_CAR_AGE_div_DAYS_EMPLOYED'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_EMPLOYED']
#app_train['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_BIRTH']
#app_train['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_EMPLOYED']
#app_train['app_DAYS_EMPLOYED_PERC'] = app_train['DAYS_EMPLOYED'] - app_train['DAYS_BIRTH']
#app_train['app_DAYS_EMPLOYED_div_DAYS_BIRTH'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']
#app_train['app_CNT_CHILDREN_div_CNT_FAM_MEMBERS'] = app_train['CNT_CHILDREN'] / app_train['CNT_FAM_MEMBERS']

### 
app_train.drop('ORGANIZATION_TYPE', axis=1, inplace=True)

len(app_train.columns)

313

In [19]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [20]:
baseline = Baseline('SK_ID_CURR')

params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 311) (61503, 311)
[100]	training's auc: 0.788047	valid_1's auc: 0.761995
[200]	training's auc: 0.817039	valid_1's auc: 0.766161
[300]	training's auc: 0.837698	valid_1's auc: 0.766059
[auc] tr:0.820701, va0.7665
-------------------- 1 --------------------
(246009, 311) (61502, 311)
[100]	training's auc: 0.788721	valid_1's auc: 0.762933
[200]	training's auc: 0.818369	valid_1's auc: 0.767284
[300]	training's auc: 0.840599	valid_1's auc: 0.767676
[400]	training's auc: 0.85772	valid_1's auc: 0.767266
[auc] tr:0.840997, va0.7677
-------------------- 2 --------------------
(246009, 311) (61502, 311)
[100]	training's auc: 0.788278	valid_1's auc: 0.766665
[200]	training's auc: 0.817396	valid_1's auc: 0.76983
[300]	training's auc: 0.83833	valid_1's auc: 0.770084
[400]	training's auc: 0.856791	valid_1's auc: 0.77015
[auc] tr:0.844076, va0.7704
-------------------- 3 --------------------
(246009, 311) (61502, 311)
[100]	training's auc: 0.789732	

In [21]:
imp.sort_values("imp", ascending=False)[:10]

,col,imp,imp_std
48,EXT_SOURCE_mean,97640.144096,3497.645727
160,NEW_EXT_SOURCES_GEO,13908.610628,3607.634094
154,NEW_CREDIT_TO_ANNUITY_RATIO,10995.574638,1458.505714
10,ANNUITY_div_CREDIT,8731.881764,847.169350
155,NEW_CREDIT_TO_GOODS_RATIO,7673.033989,418.434089
45,EXT_SOURCE_3,5548.943578,661.486282
295,app_EXT_SOURCE_1_PER_DAYS_BIRTH,5337.940505,458.748556
301,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4828.042793,866.165092
291,app_DAYS_EMPLOYED_PERC,4238.824700,510.125243
29,DAYS_ID_PUBLISH,4085.645327,720.613950


In [22]:
app_test['DAYS_EMPLOYED'] = app_test['DAYS_EMPLOYED'].replace(365243, np.nan)

In [23]:
docs = [_f for _f in app_test.columns if 'FLAG_DOC' in _f]
live = [_f for _f in app_test.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
app_test['ORGANIZATION_TYPE'] = app_test_origin['ORGANIZATION_TYPE'].copy()

# nullに置き換え
app_test['DAYS_EMPLOYED'] = app_test['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = app_test[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
app_test['INCOME_div_PERSON'] = app_test['AMT_INCOME_TOTAL'] / app_test['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
app_test['INCOME_div_EMPLOYED'] = app_test['AMT_INCOME_TOTAL'] / app_test['DAYS_EMPLOYED']

#
app_test['NEW_CREDIT_TO_ANNUITY_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_ANNUITY']

app_test['NEW_CREDIT_TO_GOODS_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_GOODS_PRICE']

app_test['NEW_DOC_IND_KURT'] = app_test[docs].kurtosis(axis=1)

app_test['NEW_LIVE_IND_SUM'] = app_test[live].sum(axis=1)

app_test['NEW_INC_PER_CHLD'] = app_test['AMT_INCOME_TOTAL'] / (1 + app_test['CNT_CHILDREN'])

app_test['NEW_INC_BY_ORG'] = app_test['ORGANIZATION_TYPE'].map(inc_by_org)

app_test['NEW_EMPLOY_TO_BIRTH_RATIO'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

app_test['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = app_test['DAYS_EMPLOYED'] / (app_test['DAYS_BIRTH'] + 18*365)

app_test['NEW_BIRTH_TO_EMPLOY_RATIO'] = app_test['DAYS_BIRTH'] / (1 + app_test['DAYS_EMPLOYED'])

app_test['NEW_INCOME_TO_ANNUITY_RATIO'] = app_test['AMT_INCOME_TOTAL'] / (1 + app_test['AMT_ANNUITY'])

app_test['NEW_ANNUITY_TO_INCOME_RATIO'] = app_test['AMT_ANNUITY'] / (1 + app_test['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
app_test['NEW_EXT_SOURCES_MEDIAN'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

app_test['EXT_SOURCE_mean'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
app_test['EXT_SOURCE_max'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

app_test['EXT_SOURCE_min'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_test['EXT_SOURCE_std'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

app_test['EXT_SOURCE_count'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

app_test['NEW_EXT_SOURCES_PROD'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

app_test['NEW_EXT_SOURCES_MAD'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

app_test['NEW_EXT_SOURCES_GEO'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
app_test['DAYS_EMPLOYED_div_BERTH'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
app_test['ANNUITY_div_INCOME'] = app_test['AMT_ANNUITY'] / app_test['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
app_test['ANNUITY_div_CREDIT'] = app_test['AMT_ANNUITY'] / app_test['AMT_CREDIT']

###
app_test['credit_goods_price_ratio'] = app_test['AMT_CREDIT'] / app_test['AMT_GOODS_PRICE']
 
app_test['credit_downpayment'] = app_test['AMT_GOODS_PRICE'] - app_test['AMT_CREDIT']

app_test['USER_AGE'] = abs(app_test['DAYS_BIRTH'])/364.25

app_test['NEW_CAR_TO_BIRTH_RATIO'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_BIRTH']

app_test['NEW_CAR_TO_EMPLOY_RATIO'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_EMPLOYED']

app_test['NEW_PHONE_TO_BIRTH_RATIO'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_BIRTH']

app_test['NEW_PHONE_TO_EMPLOYED_RATIO'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_EMPLOYED']

app_test['NEW_CREDIT_TO_INCOME_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_INCOME_TOTAL']

app_test['NEW_INCOME_CREDIT_PERC'] = app_test['AMT_INCOME_TOTAL'] / app_test['AMT_CREDIT']

app_test['NEW_INCOME_PER_PERSON'] = app_test['AMT_INCOME_TOTAL'] / app_test['CNT_FAM_MEMBERS']

### 2022/06/25
app_test['app_EXT_SOURCE_prod'] = app_test['EXT_SOURCE_1'] * app_test['EXT_SOURCE_2'] * app_test['EXT_SOURCE_3']
app_test['app_EXT_SOURCE_1_times_EXT_SOURCE_2'] = app_test['EXT_SOURCE_1'] * app_test['EXT_SOURCE_2']
app_test['app_EXT_SOURCE_1_times_EXT_SOURCE_3'] = app_test['EXT_SOURCE_1'] * app_test['EXT_SOURCE_3']
app_test['app_EXT_SOURCE_2_times_EXT_SOURCE_3'] = app_test['EXT_SOURCE_2'] * app_test['EXT_SOURCE_3']
app_test['app_EXT_SOURCE_1_times_DAYS_EMPLOYED'] = app_test['EXT_SOURCE_1'] * app_test['DAYS_EMPLOYED']
app_test['app_EXT_SOURCE_2_times_DAYS_EMPLOYED'] = app_test['EXT_SOURCE_2'] * app_test['DAYS_EMPLOYED']
app_test['app_EXT_SOURCE_3_times_DAYS_EMPLOYED'] = app_test['EXT_SOURCE_3'] * app_test['DAYS_EMPLOYED']
app_test['app_EXT_SOURCE_1_PER_DAYS_BIRTH'] = app_test['EXT_SOURCE_1'] / app_test['DAYS_BIRTH']
app_test['app_EXT_SOURCE_2_PER_DAYS_BIRTH'] = app_test['EXT_SOURCE_2'] / app_test['DAYS_BIRTH']
app_test['app_EXT_SOURCE_3_PER_DAYS_BIRTH'] = app_test['EXT_SOURCE_3'] / app_test['DAYS_BIRTH']

### 2022/06/26
#app_test['app_most_popular_AMT_GOODS_PRICE'] = app_test['AMT_GOODS_PRICE'] \
#    .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
#app_test['app_popular_AMT_GOODS_PRICE'] = app_test['AMT_GOODS_PRICE'] \
#    .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
#app_test['app_OWN_CAR_AGE_div_DAYS_BIRTH'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_BIRTH']
#app_test['app_OWN_CAR_AGE_div_DAYS_EMPLOYED'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_EMPLOYED']
#app_test['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_BIRTH']
#app_test['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_EMPLOYED']
#app_test['app_DAYS_EMPLOYED_PERC'] = app_test['DAYS_EMPLOYED'] - app_test['DAYS_BIRTH']
#app_test['app_DAYS_EMPLOYED_div_DAYS_BIRTH'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']
#app_test['app_CNT_CHILDREN_div_CNT_FAM_MEMBERS'] = app_test['CNT_CHILDREN'] / app_test['CNT_FAM_MEMBERS']

### 
app_test.drop('ORGANIZATION_TYPE', axis=1, inplace=True)

len(app_test.columns)

312

In [24]:
# データセット
x_test = app_test.drop(columns=['SK_ID_CURR'])
id_test = app_test[['SK_ID_CURR']]

for col in x_test.columns:
    if x_test[col].dtype=='O':
        x_test[col] = x_test[col].astype('category')

In [25]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [26]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036165
1,100005,0.083702
2,100013,0.017059
3,100028,0.035244
4,100038,0.157986


### CSVの読み込みと前処理

In [27]:
# infをnanへ置換
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,app_EXT_SOURCE_3_PER_DAYS_BIRTH,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,-0.000015,0,0,NaN,NaN,0.119860,1.780220,8824.0,0.067329,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,NaN,0,0,NaN,NaN,0.049389,0.696970,15577.0,0.070862,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,-0.000038,0,0,-0.001365,-0.115556,0.042791,3.622222,18821.0,0.011814,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,NaN,0,0,NaN,NaN,0.032465,0.203027,15966.0,0.159905,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,NaN,0,0,NaN,NaN,0.055489,0.364055,16894.0,0.152418,0.0


In [28]:
# 欠損値補完
app_train = app_train.fillna(-99999999)
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,app_EXT_SOURCE_3_PER_DAYS_BIRTH,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,-1.473161e-05,0,0,-1.000000e+08,-1.000000e+08,0.119860,1.780220,8824.0,0.067329,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,-1.000000e+08,0,0,-1.000000e+08,-1.000000e+08,0.049389,0.696970,15577.0,0.070862,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,-3.830551e-05,0,0,-1.365116e-03,-1.155556e-01,0.042791,3.622222,18821.0,0.011814,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,-1.000000e+08,0,0,-1.000000e+08,-1.000000e+08,0.032465,0.203027,15966.0,0.159905,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,-1.000000e+08,0,0,-1.000000e+08,-1.000000e+08,0.055489,0.364055,16894.0,0.152418,0.0


In [29]:
# infをnanへ置換
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,app_EXT_SOURCE_3_PER_DAYS_BIRTH,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,-0.000008,1,0,NaN,NaN,0.090432,0.747102,16912.0,0.121044,0.000000
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,-0.000024,0,0,NaN,NaN,-0.000000,-0.000000,13595.0,0.247398,0.000000
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,-0.000030,0,0,-0.000250,-0.001122,0.042719,0.192014,15580.0,0.222477,0.000000
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,-0.000044,0,1,NaN,NaN,0.129150,0.967310,12110.0,0.133515,0.500000
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,NaN,0,0,-0.001227,-0.007303,0.062960,0.374715,10849.0,0.168021,0.333333


In [30]:
# 欠損値補完
app_test = app_test.fillna(-99999999)
app_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,app_EXT_SOURCE_3_PER_DAYS_BIRTH,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,-8.290606e-06,1,0,-1.000000e+08,-1.000000e+08,0.090432,0.747102,16912.0,0.121044,0.000000
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,-2.396821e-05,0,0,-1.000000e+08,-1.000000e+08,-0.000000,-0.000000,13595.0,0.247398,0.000000
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,-3.049163e-05,0,0,-2.495259e-04,-1.121579e-03,0.042719,0.192014,15580.0,0.222477,0.000000
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,-4.383974e-05,0,1,-1.000000e+08,-1.000000e+08,0.129150,0.967310,12110.0,0.133515,0.500000
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,-1.000000e+08,0,0,-1.226994e-03,-7.302602e-03,0.062960,0.374715,10849.0,0.168021,0.333333


In [31]:
#app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
#app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custom data
#app_train_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_loan_from_hc_only_cash.csv')
#app_test_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_loan_from_hc_only_cash.csv')
#app_train_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_test_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_train_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_annuity_to_max_installment_ratio.csv')
#app_test_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_annuity_to_max_installment_ratio.csv')

In [32]:
#app_train = app_train_origin.copy()
#app_test = app_test_origin.copy()
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
credit_card_balance = credit_card_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
#app_train_1000_mean = app_train_1000_mean_origin.copy()
#app_test_1000_mean = app_test_1000_mean_origin.copy()
#app_train_max_installment_ratio = app_train_max_installment_ratio_origin.copy()
#app_test_max_installment_ratio = app_test_max_installment_ratio_origin.copy()
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [33]:
# infをnanへ置換
bureau = bureau.replace([np.inf, -np.inf], np.nan)
bureau_balance = bureau_balance.replace([np.inf, -np.inf], np.nan)
pos_cash_balance = pos_cash_balance.replace([np.inf, -np.inf], np.nan)
credit_card_balance = credit_card_balance.replace([np.inf, -np.inf], np.nan)
previous_application = previous_application.replace([np.inf, -np.inf], np.nan)
installments_payments = installments_payments.replace([np.inf, -np.inf], np.nan)

pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


### pos_cashの紐づけ

In [34]:
baseline = Baseline('SK_ID_CURR')

pos = pos_cash_balance.copy()
pos = baseline.reduce_mem_usage(pos)
print(pos.shape)
pos.head()

Memory usage of dataframe is 610.43 MB
Memory usage of optimization is 290039510.00 MB
Decreased by -47513511.5%
(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [35]:
pos_ohe = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'], dummy_na=True)
col_ohe = sorted(list(set(pos_ohe.columns) - set(pos.columns)))
print(len(col_ohe))
col_ohe

10


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Amortized debt',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Canceled',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Returned to the store',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_XNA',
 'NAME_CONTRACT_STATUS_nan']

In [36]:
pos_ohe_agg = pos_ohe.groupby('SK_ID_CURR').agg({
    "MONTHS_BALANCE": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT_FUTURE": ["mean", "std", "min", "max"]
    ,"SK_DPD": ["mean", "std", "min", "max"]
    ,"SK_DPD_DEF": ["mean", "std", "min", "max"]
    # カテゴリ変数をone-hot-encodingした値の集約
    ,"NAME_CONTRACT_STATUS_Active": ["mean"]
    ,"NAME_CONTRACT_STATUS_Amortized debt": ["mean"]
    ,"NAME_CONTRACT_STATUS_Approved": ["mean"]
    ,"NAME_CONTRACT_STATUS_Canceled": ["mean"]
    ,"NAME_CONTRACT_STATUS_Completed": ["mean"]
    ,"NAME_CONTRACT_STATUS_Demand": ["mean"]
    ,"NAME_CONTRACT_STATUS_Returned to the store": ["mean"]
    ,"NAME_CONTRACT_STATUS_Signed": ["mean"]
    ,"NAME_CONTRACT_STATUS_XNA": ["mean"]
    ,"NAME_CONTRACT_STATUS_nan": ["mean"]
    # IDのユニーク数をカウント
    ,"SK_ID_PREV":["count", "nunique"]
})

# カラム名の付与
pos_ohe_agg.columns = [i + "_" + j for i,j in pos_ohe_agg.columns]
pos_ohe_agg = pos_ohe_agg.reset_index(drop=False)

print(pos_ohe_agg.shape)
pos_ohe_agg.head()

(337252, 33)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [37]:
### infをnanへ置換
pos_ohe_agg = pos_ohe_agg.replace([np.inf, -np.inf], np.nan)
pos_ohe_agg.head()

,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [38]:
df_train = pd.merge(app_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 345)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [39]:
### テーブル結合
df_test = pd.merge(app_test, pos_ohe_agg, on='SK_ID_CURR', how="left")
print(df_test.shape)
df_test.head()

(48744, 344)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [40]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [41]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [42]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [43]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [44]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [45]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 802.37 MB
Memory usage of optimization is 423442775.00 MB
Decreased by -52773634.2%
Memory usage of dataframe is 126.81 MB
Memory usage of optimization is 67071872.00 MB
Decreased by -52890000.5%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [46]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [47]:
baseline = Baseline('SK_ID_CURR')

params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 343) (61503, 343)
[100]	training's auc: 0.793477	valid_1's auc: 0.766753
[200]	training's auc: 0.822583	valid_1's auc: 0.771474
[300]	training's auc: 0.84369	valid_1's auc: 0.772376
[400]	training's auc: 0.860576	valid_1's auc: 0.771991
[auc] tr:0.848980, va0.7726
-------------------- 1 --------------------
(246009, 343) (61502, 343)
[100]	training's auc: 0.7932	valid_1's auc: 0.767917
[200]	training's auc: 0.822258	valid_1's auc: 0.773751
[300]	training's auc: 0.844106	valid_1's auc: 0.774999
[400]	training's auc: 0.862332	valid_1's auc: 0.775524
[500]	training's auc: 0.877578	valid_1's auc: 0.775633
[auc] tr:0.872893, va0.7759
-------------------- 2 --------------------
(246009, 343) (61502, 343)
[100]	training's auc: 0.792989	valid_1's auc: 0.771619
[200]	training's auc: 0.821908	valid_1's auc: 0.775555
[300]	training's auc: 0.842948	valid_1's auc: 0.776205
[400]	training's auc: 0.860768	valid_1's auc: 0.775791
[auc] tr:0.846856, 

In [48]:
imp.sort_values("imp", ascending=False)[:10]

,col,imp,imp_std
56,EXT_SOURCE_mean,98070.984690,3444.439086
182,NEW_EXT_SOURCES_GEO,12620.975072,3439.730378
176,NEW_CREDIT_TO_ANNUITY_RATIO,11001.462345,885.587456
10,ANNUITY_div_CREDIT,6683.278529,869.123190
21,CNT_INSTALMENT_FUTURE_mean,6568.095568,652.153818
177,NEW_CREDIT_TO_GOODS_RATIO,6547.867335,283.660933
123,MONTHS_BALANCE_std,5706.954909,620.268632
53,EXT_SOURCE_3,5472.798400,599.436547
335,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,4842.705026,338.186262
333,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4695.217271,816.579071


In [49]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [50]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [51]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.030709
1,100005,0.091537
2,100013,0.017933
3,100028,0.035646
4,100038,0.174762


In [52]:
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}


def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [53]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [54]:
#baseline = Baseline('SK_ID_CURR')

#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [55]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [56]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [57]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [58]:
# 推論用データセットの作成
#x_test = df_test.drop(columns=['SK_ID_CURR'])
#id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

# predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

# make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuning.csv", index=None)

### ### credit card blanceの読み込み

In [59]:
credit = credit_card_balance_origin.copy()
cerdit = baseline.reduce_mem_usage(credit)
print(credit.shape)
credit.head()

Memory usage of dataframe is 673.88 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -49579318.7%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [60]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in credit.columns:
    if credit[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

NAME_CONTRACT_STATUS


['NAME_CONTRACT_STATUS']

In [61]:
# one-hot-encoding
for col in encoding_columns:
    credit_ohe = pd.get_dummies(credit, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(credit_ohe.columns) - set(credit.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [62]:
print(credit.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')


In [63]:
credit_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'SK_ID_CURR': ['mean', 'std', 'min', 'max']
        ,'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
credit_ohe_agg.columns = [i + "_" + j for i,j in credit_ohe_agg.columns]
credit_ohe_agg = credit_ohe_agg.reset_index(drop=False)

print(credit_ohe_agg.shape)
credit_ohe_agg.head()

(103558, 95)


,SK_ID_CURR,SK_ID_CURR_mean,SK_ID_CURR_std,SK_ID_CURR_min,SK_ID_CURR_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,100006.0,0.0,100006,100006,-3.5,1.870829,-6,-1,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,100011.0,0.0,100011,100011,-38.5,21.505813,-75,-2,54482.109375,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,100013.0,0.0,100013,100013,-48.5,27.856777,-96,-1,18159.919922,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,100021.0,0.0,100021,100021,-10.0,5.049752,-18,-2,0.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,100023.0,0.0,100023,100023,-7.5,2.449490,-11,-4,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [64]:
# infをnanへ置換
credit_ohe_agg = credit_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [65]:
df_train = pd.merge(app_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 439)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [66]:
# テーブル結合
df_test = pd.merge(app_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 438)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [67]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [68]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [69]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [70]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [71]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [72]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 970.12 MB
Memory usage of optimization is 539066911.00 MB
Decreased by -55566826.6%
Memory usage of dataframe is 153.40 MB
Memory usage of optimization is 85399616.00 MB
Decreased by -55669792.2%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [73]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [74]:
baseline = Baseline('SK_ID_CURR')

params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 437) (61503, 437)
[100]	training's auc: 0.79729	valid_1's auc: 0.768638
[200]	training's auc: 0.825315	valid_1's auc: 0.774498
[300]	training's auc: 0.846482	valid_1's auc: 0.775174
[auc] tr:0.843231, va0.7753
-------------------- 1 --------------------
(246009, 437) (61502, 437)
[100]	training's auc: 0.796069	valid_1's auc: 0.771651
[200]	training's auc: 0.824637	valid_1's auc: 0.77753
[300]	training's auc: 0.846054	valid_1's auc: 0.779144
[400]	training's auc: 0.863301	valid_1's auc: 0.779532
[500]	training's auc: 0.877854	valid_1's auc: 0.779591
[600]	training's auc: 0.8906	valid_1's auc: 0.779692
[700]	training's auc: 0.901501	valid_1's auc: 0.780136
[800]	training's auc: 0.911204	valid_1's auc: 0.779901
[auc] tr:0.905492, va0.7802
-------------------- 2 --------------------
(246009, 437) (61502, 437)
[100]	training's auc: 0.795936	valid_1's auc: 0.773394
[200]	training's auc: 0.824985	valid_1's auc: 0.777888
[300]	training's auc

In [75]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
124,EXT_SOURCE_mean,97542.204894,3477.598642
262,NEW_EXT_SOURCES_GEO,12942.182380,2982.286582
256,NEW_CREDIT_TO_ANNUITY_RATIO,10383.761899,1157.599835
85,CNT_INSTALMENT_FUTURE_mean,6402.655028,1200.553381
58,ANNUITY_div_CREDIT,6177.937551,1153.362144
257,NEW_CREDIT_TO_GOODS_RATIO,6165.778134,517.978205
195,MONTHS_BALANCE_std_y,5547.621171,1083.326402
121,EXT_SOURCE_3,5291.320898,909.522180
427,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4804.472799,1003.849554
429,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,4770.437190,834.020000


In [76]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [77]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [78]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering3.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036888
1,100005,0.086799
2,100013,0.025257
3,100028,0.059578
4,100038,0.166539


### previous applicationの読み込み

In [79]:
previous = previous_application_origin.copy()
previous = baseline.reduce_mem_usage(credit)
print(previous.shape)
previous.head()

Memory usage of dataframe is 318.63 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -104857500.0%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [80]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in previous.columns:
    if previous[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_STATUS


In [81]:
# one-hot-encoding
for col in encoding_columns:
    previous_ohe = pd.get_dummies(previous, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(previous_ohe.columns) - set(previous.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [82]:
print(previous.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')


In [83]:
previous_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
previous_ohe_agg.columns = [i + "_" + j for i,j in previous_ohe_agg.columns]
previous_ohe_agg = previous_ohe_agg.reset_index(drop=False)

print(previous_ohe_agg.shape)
previous_ohe_agg.head()

(103558, 91)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,AMT_BALANCE_std,AMT_BALANCE_min,AMT_BALANCE_max,AMT_CREDIT_LIMIT_ACTUAL_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,-3.5,1.870829,-6,-1,0.000000,0.000000,0.0,0.00000,270000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,-38.5,21.505813,-75,-2,54482.109375,68127.238133,0.0,189000.00000,164189.189189,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,-48.5,27.856777,-96,-1,18159.919922,43237.406465,0.0,161420.21875,131718.750000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,-10.0,5.049752,-18,-2,0.000000,0.000000,0.0,0.00000,675000.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,-7.5,2.449490,-11,-4,0.000000,0.000000,0.0,0.00000,135000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [84]:
# infをnanへ置換
previous_ohe_agg = previous_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [85]:
df_train = pd.merge(app_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 529)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [86]:
# テーブル結合
df_test = pd.merge(app_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 528)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [87]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [88]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [89]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [90]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [91]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [92]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1128.49 MB
Memory usage of optimization is 649770871.00 MB
Decreased by -57578935.5%
Memory usage of dataframe is 178.51 MB
Memory usage of optimization is 102947456.00 MB
Decreased by -57671612.3%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [93]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [94]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 527) (61503, 527)
[100]	training's auc: 0.79729	valid_1's auc: 0.768638
[200]	training's auc: 0.825315	valid_1's auc: 0.774498
[300]	training's auc: 0.846482	valid_1's auc: 0.775174
[auc] tr:0.843231, va0.7753
-------------------- 1 --------------------
(246009, 527) (61502, 527)
[100]	training's auc: 0.796069	valid_1's auc: 0.771651
[200]	training's auc: 0.824637	valid_1's auc: 0.77753
[300]	training's auc: 0.846054	valid_1's auc: 0.779144
[400]	training's auc: 0.863301	valid_1's auc: 0.779532
[500]	training's auc: 0.877854	valid_1's auc: 0.779591
[600]	training's auc: 0.8906	valid_1's auc: 0.779692
[700]	training's auc: 0.901501	valid_1's auc: 0.780136
[800]	training's auc: 0.911204	valid_1's auc: 0.779901
[auc] tr:0.905492, va0.7802
-------------------- 2 --------------------
(246009, 527) (61502, 527)
[100]	training's auc: 0.795936	valid_1's auc: 0.773394
[200]	training's auc: 0.824985	valid_1's auc: 0.777888
[300]	training's auc

In [95]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
192,EXT_SOURCE_mean,97476.295702,3461.105470
342,NEW_EXT_SOURCES_GEO,12900.086354,2981.271351
336,NEW_CREDIT_TO_ANNUITY_RATIO,10284.896998,1014.665025
149,CNT_INSTALMENT_FUTURE_mean,6281.885185,1097.761431
337,NEW_CREDIT_TO_GOODS_RATIO,6098.754066,453.251804
106,ANNUITY_div_CREDIT,6080.010642,1160.549972
265,MONTHS_BALANCE_std,5403.517865,911.836206
189,EXT_SOURCE_3,5210.411669,884.958323
517,app_EXT_SOURCE_2_times_EXT_SOURCE_3,4763.707145,930.186503
519,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,4711.690057,807.343641


In [96]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [97]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [98]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering4.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036821
1,100005,0.087740
2,100013,0.027015
3,100028,0.057691
4,100038,0.166455


### installments paymentsの読み込み

In [99]:
installments = installments_payments_origin.copy()
installments = baseline.reduce_mem_usage(installments)
print(installments.shape)
installments.head()

Memory usage of dataframe is 830.41 MB
Memory usage of optimization is 408162158.00 MB
Decreased by -49151908.2%
(13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [100]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in installments.columns:
    if installments[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

[]

In [101]:
# one-hot-encoding
#for col in encoding_columns:
#    installments_ohe = pd.get_dummies(installments, columns=[col], dummy_na=True)
#    col_ohe = sorted(list(set(installments_ohe.columns) - set(installments.columns)))

#print(len(col_ohe))
#col_ohe
installments_ohe = installments.copy()

In [102]:
print(installments.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')


In [103]:
installments_ohe = installments_ohe.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_IP'})

In [104]:
installments_ohe_agg = installments_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'NUM_INSTALMENT_VERSION': ['mean', 'std', 'min', 'max']
        ,'NUM_INSTALMENT_NUMBER': ['mean', 'std', 'min', 'max']
        ,'DAYS_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENTRY_PAYMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT': ['mean', 'std', 'min', 'max']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV_IP":["count", "nunique"]
    }
)

# カラム名の付与
installments_ohe_agg.columns = [i + "_" + j for i,j in installments_ohe_agg.columns]
installments_ohe_agg = installments_ohe_agg.reset_index(drop=False)

print(installments_ohe_agg.shape)
installments_ohe_agg.head()

(339587, 27)


,SK_ID_CURR,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_std,NUM_INSTALMENT_VERSION_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_NUMBER_mean,NUM_INSTALMENT_NUMBER_std,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_NUMBER_max,DAYS_INSTALMENT_mean,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,100001,1.142857,0.377964,1.0,2.0,2.714286,1.112697,1,4,-2187.714355,...,5885.132324,5076.676763,3951.000000,17397.900391,5885.132324,5076.676763,3951.000000,17397.900391,7,2
1,100002,1.052632,0.229416,1.0,2.0,10.000000,5.627314,1,19,-295.000000,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19,1
2,100003,1.040000,0.200000,1.0,2.0,5.080000,3.134751,1,12,-1378.160034,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25,3
3,100004,1.333333,0.577350,1.0,2.0,2.000000,1.000000,1,3,-754.000000,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3,1
4,100005,1.111111,0.333333,1.0,2.0,5.000000,2.738613,1,9,-586.000000,...,6240.205078,4281.014648,4813.200195,17656.244141,6240.205078,4281.014648,4813.200195,17656.244141,9,1


In [105]:
# infをnanへ置換
installments_ohe_agg = installments_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [106]:
df_train = pd.merge(app_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 555)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [107]:
# テーブル結合
df_test = pd.merge(app_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 554)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [108]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [109]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [110]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [111]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [112]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [113]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1171.89 MB
Memory usage of optimization is 681752015.00 MB
Decreased by -58175405.2%
Memory usage of dataframe is 185.39 MB
Memory usage of optimization is 108016832.00 MB
Decreased by -58265838.8%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [114]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [115]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 553) (61503, 553)
[100]	training's auc: 0.799973	valid_1's auc: 0.77099
[200]	training's auc: 0.829667	valid_1's auc: 0.777204
[300]	training's auc: 0.851438	valid_1's auc: 0.777436
[400]	training's auc: 0.868695	valid_1's auc: 0.778073
[500]	training's auc: 0.883273	valid_1's auc: 0.777525
[auc] tr:0.872047, va0.7781
-------------------- 1 --------------------
(246009, 553) (61502, 553)
[100]	training's auc: 0.79958	valid_1's auc: 0.774276
[200]	training's auc: 0.829004	valid_1's auc: 0.780244
[300]	training's auc: 0.851436	valid_1's auc: 0.78173
[400]	training's auc: 0.869275	valid_1's auc: 0.782498
[500]	training's auc: 0.884085	valid_1's auc: 0.782597
[600]	training's auc: 0.896679	valid_1's auc: 0.782898
[700]	training's auc: 0.908037	valid_1's auc: 0.782972
[auc] tr:0.900280, va0.7832
-------------------- 2 --------------------
(246009, 553) (61502, 553)
[100]	training's auc: 0.799363	valid_1's auc: 0.776026
[200]	training's au

In [116]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
208,EXT_SOURCE_mean,97461.368403,3651.672952
358,NEW_EXT_SOURCES_GEO,12673.184617,3306.431868
352,NEW_CREDIT_TO_ANNUITY_RATIO,10062.974288,1405.748977
82,AMT_PAYMENT_min,7227.481965,1129.003113
157,CNT_INSTALMENT_FUTURE_mean,6538.527308,644.314251
353,NEW_CREDIT_TO_GOODS_RATIO,5855.780771,519.141927
114,ANNUITY_div_CREDIT,5502.830779,918.500384
205,EXT_SOURCE_3,5050.095820,722.764967
545,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,5031.174812,498.196107
543,app_EXT_SOURCE_2_times_EXT_SOURCE_3,5009.304671,556.226641


In [117]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [118]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [119]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering5.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.038825
1,100005,0.094224
2,100013,0.045140
3,100028,0.058829
4,100038,0.141545


### bureauの読み込み

In [120]:
bureau = bureau_origin.copy()
bureau = baseline.reduce_mem_usage(bureau)
print(bureau.shape)
bureau.head()

Memory usage of dataframe is 222.62 MB
Memory usage of optimization is 125299372.00 MB
Decreased by -56283785.5%
(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [121]:
### 
for col in bureau.columns:
    bureau[col] = bureau[col].replace('\s', '_', regex=True)
    bureau[col] = bureau[col].replace(':', '_', regex=True)
    bureau[col] = bureau[col].replace('/', '_', regex=True)
    
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency_1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer_credit,-131,NaN
1,215354,5714463,Active,currency_1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit_card,-20,NaN
2,215354,5714464,Active,currency_1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer_credit,-16,NaN
3,215354,5714465,Active,currency_1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit_card,-16,NaN
4,215354,5714466,Active,currency_1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer_credit,-21,NaN


In [122]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in bureau.columns:
    if bureau[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

CREDIT_ACTIVE
CREDIT_CURRENCY
CREDIT_TYPE


In [123]:
# one-hot-encoding
bureau_ohe = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_CURRENCY'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_TYPE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

print(len(col_ohe))
col_ohe

26


['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad_debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency_1',
 'CREDIT_CURRENCY_currency_2',
 'CREDIT_CURRENCY_currency_3',
 'CREDIT_CURRENCY_currency_4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another_type_of_loan',
 'CREDIT_TYPE_Car_loan',
 'CREDIT_TYPE_Cash_loan_(non-earmarked)',
 'CREDIT_TYPE_Consumer_credit',
 'CREDIT_TYPE_Credit_card',
 'CREDIT_TYPE_Interbank_credit',
 'CREDIT_TYPE_Loan_for_business_development',
 'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)',
 'CREDIT_TYPE_Loan_for_the_purchase_of_equipment',
 'CREDIT_TYPE_Loan_for_working_capital_replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile_operator_loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real_estate_loan',
 'CREDIT_TYPE_Unknown_type_of_loan',
 'CREDIT_TYPE_nan']

In [124]:
bureau_ohe = bureau_ohe.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_BUREAU'})

In [125]:
bureau_ohe.head()

,SK_ID_CURR,SK_ID_BUREAU_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan_for_business_development,CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending),CREDIT_TYPE_Loan_for_the_purchase_of_equipment,CREDIT_TYPE_Loan_for_working_capital_replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile_operator_loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real_estate_loan,CREDIT_TYPE_Unknown_type_of_loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [126]:
bureau_ohe_agg = bureau_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'DAYS_CREDIT': ['mean', 'std', 'min', 'max', 'var']
        ,'CREDIT_DAY_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_ENDDATE': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENDDATE_FACT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_MAX_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'CNT_CREDIT_PROLONG': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_DEBT': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_LIMIT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_UPDATE': ['mean', 'std', 'min', 'max']
        ,'AMT_ANNUITY': ['mean', 'std', 'min', 'max']
        # 
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        ,'CREDIT_ACTIVE_Active': ['mean']
        ,'CREDIT_ACTIVE_Bad_debt': ['mean']
        ,'CREDIT_ACTIVE_Closed': ['mean']
        ,'CREDIT_ACTIVE_Sold': ['mean']
        ,'CREDIT_ACTIVE_nan': ['mean']
        ,'CREDIT_CURRENCY_currency_1': ['mean']
        ,'CREDIT_CURRENCY_currency_2': ['mean']
        ,'CREDIT_CURRENCY_currency_3': ['mean']
        ,'CREDIT_CURRENCY_currency_4': ['mean']
        ,'CREDIT_CURRENCY_nan': ['mean']
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        # IDのユニーク数をカウント
        ,'SK_ID_BUREAU_BUREAU': ["count", "nunique"]
    }
)

# カラム名の付与
bureau_ohe_agg.columns = [i + "_" + j for i,j in bureau_ohe_agg.columns]
bureau_ohe_agg = bureau_ohe_agg.reset_index(drop=False)

#
bureau_ohe_agg['debt_credit_ratio_None'] = bureau_ohe_agg['AMT_CREDIT_SUM_DEBT_sum'] / bureau_ohe_agg['AMT_CREDIT_SUM_sum']

print(bureau_ohe_agg.shape)
bureau_ohe_agg.head()

(305811, 81)


,SK_ID_CURR,DAYS_CREDIT_mean,DAYS_CREDIT_std,DAYS_CREDIT_min,DAYS_CREDIT_max,DAYS_CREDIT_var,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_std,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_max,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,100001,-735.000000,489.942514,-1572,-49,240043.666667,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,7,0.410555
1,100002,-874.000000,431.451040,-1437,-103,186150.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,8,0.284122
2,100003,-1400.750000,909.826128,-2586,-606,827783.583333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,4,0.000000
3,100004,-867.000000,649.124025,-1326,-408,421362.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,2,0.000000
4,100005,-190.666667,162.297053,-373,-62,26340.333333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,3,0.864992


In [127]:
# infをnanへ置換
bureau_ohe_agg = bureau_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [128]:
df_train = pd.merge(app_train, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 635)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [129]:
df_test = pd.merge(app_test, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 634)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [130]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('EXT_SOURCE_3')
encoding_columns.remove('EXT_SOURCE_2')
encoding_columns.remove('EXT_SOURCE_1')

In [131]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [132]:
encoding_columns.append('EXT_SOURCE_3')
encoding_columns.append('EXT_SOURCE_2')
encoding_columns.append('EXT_SOURCE_1')

In [133]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [134]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [135]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [136]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()


Memory usage of dataframe is 1359.58 MB
Memory usage of optimization is 780155535.00 MB
Decreased by -57382072.3%
Memory usage of dataframe is 215.14 MB
Memory usage of optimization is 123614912.00 MB
Decreased by -57458629.1%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [137]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [138]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 633) (61503, 633)
[100]	training's auc: 0.804363	valid_1's auc: 0.772836
[200]	training's auc: 0.835128	valid_1's auc: 0.780577
[300]	training's auc: 0.858961	valid_1's auc: 0.781669
[400]	training's auc: 0.877321	valid_1's auc: 0.781954
[500]	training's auc: 0.892132	valid_1's auc: 0.782417
[600]	training's auc: 0.904883	valid_1's auc: 0.782694
[700]	training's auc: 0.916173	valid_1's auc: 0.782605
[800]	training's auc: 0.92545	valid_1's auc: 0.782473
[auc] tr:0.920909, va0.7829
-------------------- 1 --------------------
(246009, 633) (61502, 633)
[100]	training's auc: 0.803624	valid_1's auc: 0.776103
[200]	training's auc: 0.835009	valid_1's auc: 0.784168
[300]	training's auc: 0.858214	valid_1's auc: 0.785864
[400]	training's auc: 0.876828	valid_1's auc: 0.786384
[500]	training's auc: 0.89164	valid_1's auc: 0.786423
[auc] tr:0.887030, va0.7868
-------------------- 2 --------------------
(246009, 633) (61502, 633)
[100]	training's a

In [139]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
285,EXT_SOURCE_mean,97577.405496,2763.432715
435,NEW_EXT_SOURCES_GEO,13824.853232,3114.914124
429,NEW_CREDIT_TO_ANNUITY_RATIO,8724.043532,1115.262523
108,AMT_PAYMENT_min,7032.726567,305.272701
187,CNT_INSTALMENT_FUTURE_mean,6065.650785,584.263787
632,debt_credit_ratio_None,5672.513309,578.146243
430,NEW_CREDIT_TO_GOODS_RATIO,5565.649577,122.273274
140,ANNUITY_div_CREDIT,5430.925656,381.534212
247,DAYS_CREDIT_max,4814.217715,606.704348
624,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,4448.898235,462.626954


In [140]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [141]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [142]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering6.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.041011
1,100005,0.120685
2,100013,0.034984
3,100028,0.058334
4,100038,0.143632


### mean mean 

In [155]:
# consumer_annuity_to_max_installment_ratio
app_train_mean1 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_test_mean1 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_train_mean1 = app_train_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean1 = app_test_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

### installment_payment_ratio_1000_mean_mean
#### consumer_installment_payment_ratio_1000_mean_mean
app_train_mean2 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean2 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean2 = app_train_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean2 = app_test_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


#### revo_installment_payment_ratio_1000_mean_mean
app_train_mean3 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean3 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean3 = app_train_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean3 = app_test_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

#### installment_payment_ratio_1000_mean_mean
app_train_mean4 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean4 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean4 = app_train_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean4 = app_test_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


In [156]:
# infをnanへ置換
app_train_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)
app_test_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)

app_train_mean2 = app_train_mean2.replace([np.inf, -np.inf], np.nan)
app_test_mean2 = app_test_mean2.replace([np.inf, -np.inf], np.nan)

app_train_mean3 = app_train_mean3.replace([np.inf, -np.inf], np.nan)
app_test_mean3 = app_test_mean3.replace([np.inf, -np.inf], np.nan)

app_train_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)
app_test_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)

In [157]:
#
#app_train_mean1 = app_train_mean1.fillna(0)
#app_test_mean1 = app_test_mean1.fillna(0)

#app_train_mean2 = app_train_mean2.fillna(0)
#app_test_mean2 = app_test_mean2.fillna(0)

#app_train_mean3 = app_train_mean3.fillna(0)
#app_test_mean3 = app_test_mean3.fillna(0)

#app_train_mean4 = app_train_mean4.fillna(0)
#app_test_mean4 = app_test_mean4.fillna(0)

In [158]:
df_train = pd.merge(app_train, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')

df_train = pd.merge(df_train, app_train_mean1, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean2, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean3, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean4, on='SK_ID_CURR', how='left')

df_train = df_train.fillna(0)

print(df_train.shape)
df_train.head()

(307511, 639)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.000000,0.000000,0.0,0.0,19.0,1.0,-988.0000,0.0,0.0,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.000000,0.000000,0.0,0.0,28.0,3.0,0.0000,0.0,0.0,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.000000,0.000000,0.0,0.0,4.0,1.0,0.0000,0.0,0.0,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.047619,0.000000,0.0,0.0,21.0,3.0,-2698.7273,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.000000,0.015152,0.0,0.0,66.0,5.0,0.0000,0.0,0.0,0.0


In [159]:
df_test = pd.merge(app_test, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')

df_test = pd.merge(df_test, app_test_mean1, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean2, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean3, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean4, on='SK_ID_CURR', how='left')

df_test = df_test.fillna(0)

print(df_test.shape)
df_test.head()

(48744, 638)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.000000,0.0,0.0,9.0,2.0,0.0,0.00000,0.000000,0.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.090909,0.0,0.0,11.0,1.0,0.0,0.00000,0.000000,0.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.027778,0.0,0.0,36.0,3.0,0.0,-1185.60375,0.000000,0.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.000000,0.0,0.0,31.0,2.0,0.0,0.00000,-1148.027459,0.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.000000,0.0,0.0,13.0,1.0,0.0,0.00000,0.000000,0.0


In [160]:
# 処理対象の列名をリスト化(Rev/Cashのcolumnを取得する)
replace_columns = list(df_test.select_dtypes(include='number').columns)
replace_columns.remove('SK_ID_CURR')
replace_columns.remove('EXT_SOURCE_3')
replace_columns.remove('EXT_SOURCE_2')
replace_columns.remove('EXT_SOURCE_1')

In [161]:
### 欠損値を補完する

#for column in processing_columns:
#    #print("Missing Value Completion Process: " + column + "\n")
#    result = modules.process_imputer(df_train, df_test, replace_columns, column)
#    if result != True:
#        print("Can not Process: " + column + "\n")

In [162]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
#encoding_columns.remove('EXT_SOURCE_3')
#encoding_columns.remove('EXT_SOURCE_2')
#encoding_columns.remove('EXT_SOURCE_1')

In [163]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.118523,-0.476503,-0.00255,0.0,-0.118159,-0.856749,-3.049664,0.086846,0.222678,0.011144
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.118523,-0.476503,-0.00255,0.0,0.298617,0.462282,0.328005,0.086846,0.222678,0.011144
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.118523,-0.476503,-0.00255,0.0,-1.311051,-0.856749,0.328005,0.086846,0.222678,0.011144
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,8.437188,-0.476503,-0.00255,0.0,-0.015781,0.462282,-3.049843,0.086846,0.222678,0.011144
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.118523,1.595367,-0.00255,0.0,1.433840,1.232306,0.328005,0.086846,0.222678,0.011144


In [164]:
#encoding_columns.append('EXT_SOURCE_3')
#encoding_columns.append('EXT_SOURCE_2')
#encoding_columns.append('EXT_SOURCE_1')

In [165]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.118523,-0.476503,-0.00255,0.0,-0.118159,-0.856749,-3.049664,0.086846,0.222678,0.011144
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.118523,-0.476503,-0.00255,0.0,0.298617,0.462282,0.328005,0.086846,0.222678,0.011144
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.118523,-0.476503,-0.00255,0.0,-1.311051,-0.856749,0.328005,0.086846,0.222678,0.011144
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,8.437188,-0.476503,-0.00255,0.0,-0.015781,0.462282,-3.049843,0.086846,0.222678,0.011144
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.118523,1.595367,-0.00255,0.0,1.433840,1.232306,0.328005,0.086846,0.222678,0.011144


In [166]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.118523,-0.476503,-0.00255,0.0,-0.118159,-0.856749,-3.049664,0.086846,0.222678,0.011144
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.118523,-0.476503,-0.00255,0.0,0.298617,0.462282,0.328005,0.086846,0.222678,0.011144
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.118523,-0.476503,-0.00255,0.0,-1.311051,-0.856749,0.328005,0.086846,0.222678,0.011144
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,8.437188,-0.476503,-0.00255,0.0,-0.015781,0.462282,-3.049843,0.086846,0.222678,0.011144
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.118523,1.595367,-0.00255,0.0,1.433840,1.232306,0.328005,0.086846,0.222678,0.011144


In [167]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.118523,-0.476503,-0.00255,0.0,-0.118159,-0.856749,-3.049664,0.086846,0.222678,0.011144
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.118523,-0.476503,-0.00255,0.0,0.298617,0.462282,0.328005,0.086846,0.222678,0.011144
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.118523,-0.476503,-0.00255,0.0,-1.311051,-0.856749,0.328005,0.086846,0.222678,0.011144
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,8.437188,-0.476503,-0.00255,0.0,-0.015781,0.462282,-3.049843,0.086846,0.222678,0.011144
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.118523,1.595367,-0.00255,0.0,1.433840,1.232306,0.328005,0.086846,0.222678,0.011144


In [168]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1368.96 MB
Memory usage of optimization is 785075711.00 MB
Decreased by -57348116.1%
Memory usage of dataframe is 216.62 MB
Memory usage of optimization is 124394816.00 MB
Decreased by -57424088.8%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11853,-0.476562,-0.00255,0.0,-0.118164,-0.856934,-3.048828,0.086853,0.222656,0.011147
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11853,-0.476562,-0.00255,0.0,0.298584,0.462402,0.328125,0.086853,0.222656,0.011147
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11853,-0.476562,-0.00255,0.0,-1.311523,-0.856934,0.328125,0.086853,0.222656,0.011147
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,8.43750,-0.476562,-0.00255,0.0,-0.015778,0.462402,-3.050781,0.086853,0.222656,0.011147
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11853,1.595703,-0.00255,0.0,1.433594,1.232422,0.328125,0.086853,0.222656,0.011147


In [169]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [170]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 637) (61503, 637)
[100]	training's auc: 0.803425	valid_1's auc: 0.773642
[200]	training's auc: 0.835517	valid_1's auc: 0.781841
[300]	training's auc: 0.858693	valid_1's auc: 0.783387
[400]	training's auc: 0.876426	valid_1's auc: 0.783614
[500]	training's auc: 0.892102	valid_1's auc: 0.783755
[600]	training's auc: 0.90507	valid_1's auc: 0.78415
[700]	training's auc: 0.915848	valid_1's auc: 0.783943
[auc] tr:0.906877, va0.7842
-------------------- 1 --------------------
(246009, 637) (61502, 637)
[100]	training's auc: 0.803359	valid_1's auc: 0.776621
[200]	training's auc: 0.835061	valid_1's auc: 0.784921
[300]	training's auc: 0.85868	valid_1's auc: 0.786448
[400]	training's auc: 0.877224	valid_1's auc: 0.786593
[auc] tr:0.867649, va0.7868
-------------------- 2 --------------------
(246009, 637) (61502, 637)
[100]	training's auc: 0.802136	valid_1's auc: 0.778828
[200]	training's auc: 0.834798	valid_1's auc: 0.785425
[300]	training's au

In [171]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
285,EXT_SOURCE_mean,96868.024026,2542.726184
435,NEW_EXT_SOURCES_GEO,14188.956149,2810.792468
429,NEW_CREDIT_TO_ANNUITY_RATIO,9274.245735,1369.199837
187,CNT_INSTALMENT_FUTURE_mean,5780.119905,704.549445
430,NEW_CREDIT_TO_GOODS_RATIO,5767.641601,526.057934
140,ANNUITY_div_CREDIT,5409.590617,400.134865
634,debt_credit_ratio_None,4769.445637,588.235470
108,AMT_PAYMENT_min,4540.009106,734.882960
632,credit_downpayment,4470.425630,408.257787
247,DAYS_CREDIT_max,4327.050730,734.378608


In [172]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [173]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [174]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering7.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.037425
1,100005,0.138288
2,100013,0.054541
3,100028,0.045265
4,100038,0.140319


## パラメータ・チューニング

In [175]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-06-26 16:46:34,085] A new study created in memory with name: no-name-5b9c0791-2edc-43b5-bf62-4434b901df8e
[I 2022-06-26 16:49:28,560] Trial 0 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 18, 'min_child_samples': 199, 'min_sum_hessian_in_leaf': 0.0005783563056763419, 'feature_fraction': 0.6648897581255998, 'bagging_fraction': 0.6651780554340612, 'lambda_l1': 0.09115200314640104, 'lambda_l2': 1.354427914464539}. Best is trial 0 with value: 0.7842227831546568.
[I 2022-06-26 16:49:28,681] Trial 3 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 239, 'min_child_samples': 194, 'min_sum_hessian_in_leaf': 0.00034104127440856037, 'feature_fraction': 0.5415949300325582, 'bagging_fraction': 0.8937094993364361, 'lambda_l1': 26.903456603539276, 'lambda_l2': 0.09402636193643896}. Best is trial 0 with value: 0.7842227831546568.
[I 2022-06-26 16:49:31,723] Trial 4 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 91, 'min_chi

[I 2022-06-26 17:01:29,911] Trial 21 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 52, 'min_child_samples': 126, 'min_sum_hessian_in_leaf': 0.009240199198602651, 'feature_fraction': 0.6983863961647615, 'bagging_fraction': 0.645905624156174, 'lambda_l1': 0.402026007149625, 'lambda_l2': 0.7624721109559573}. Best is trial 0 with value: 0.7842227831546568.
[I 2022-06-26 17:01:45,436] Trial 23 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 53, 'min_child_samples': 117, 'min_sum_hessian_in_leaf': 0.00984762530611819, 'feature_fraction': 0.7911040387947569, 'bagging_fraction': 0.6298953391337008, 'lambda_l1': 0.2703658040940913, 'lambda_l2': 95.82542983858062}. Best is trial 0 with value: 0.7842227831546568.
[I 2022-06-26 17:02:04,127] Trial 22 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 57, 'min_child_samples': 124, 'min_sum_hessian_in_leaf': 0.001142318136738726, 'feature_fraction': 0.695413774755518, 'bagging_fract

[I 2022-06-26 17:13:49,342] Trial 40 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 119, 'min_child_samples': 189, 'min_sum_hessian_in_leaf': 0.00019544921776535888, 'feature_fraction': 0.6734108675919367, 'bagging_fraction': 0.5968938106363542, 'lambda_l1': 0.38601074071621233, 'lambda_l2': 1.3932189859763366}. Best is trial 0 with value: 0.7842227831546568.
[I 2022-06-26 17:14:13,717] Trial 43 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 139, 'min_child_samples': 85, 'min_sum_hessian_in_leaf': 0.0070621273042833655, 'feature_fraction': 0.6864244075487855, 'bagging_fraction': 0.5745652380312865, 'lambda_l1': 0.4401533924916172, 'lambda_l2': 8.556377516676989}. Best is trial 0 with value: 0.7842227831546568.
[I 2022-06-26 17:14:31,178] Trial 44 finished with value: 0.7842227831546568 and parameters: {'num_leaves': 66, 'min_child_samples': 76, 'min_sum_hessian_in_leaf': 8.794329665490216e-05, 'feature_fraction': 0.8263929950994, 'bagging

In [176]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

auc(best)=0.7842


{'num_leaves': 18,
 'min_child_samples': 199,
 'min_sum_hessian_in_leaf': 0.0005783563056763419,
 'feature_fraction': 0.6648897581255998,
 'bagging_fraction': 0.6651780554340612,
 'lambda_l1': 0.09115200314640104,
 'lambda_l2': 1.354427914464539}

In [177]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 18,
 'min_child_samples': 199,
 'min_sum_hessian_in_leaf': 0.0005783563056763419,
 'feature_fraction': 0.6648897581255998,
 'bagging_fraction': 0.6651780554340612,
 'lambda_l1': 0.09115200314640104,
 'lambda_l2': 1.354427914464539,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'learning_rate': 0.05,
 'n_estimators': 100000,
 'random_state': 123,
 'bagging_freq': 1}

In [178]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

-------------------- 0 --------------------
(246008, 637) (61503, 637)
[LightGBM] [Warning] feature_fraction is set=0.6648897581255998, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6648897581255998
[LightGBM] [Warning] lambda_l1 is set=0.09115200314640104, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.09115200314640104
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0005783563056763419, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0005783563056763419
[LightGBM] [Warning] bagging_fraction is set=0.6651780554340612, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6651780554340612
[LightGBM] [Warning] lambda_l2 is set=1.354427914464539, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.354427914464539
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[100]	training's auc: 0.78559	valid_1's auc: 0.771409
[200]	training's auc: 0

In [179]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuning.csv", index=None)

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.
(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.041140
1,100005,0.139634
2,100013,0.033028
3,100028,0.046426
4,100038,0.135516


In [180]:
del df_train
del df_test
gc.collect()

128